In [ ]:
# dbutils.widgets.text("step_id","18111") dbks2mysql
# dbutils.widgets.text("step_id","18232") # redshift2mysql
# dbutils.widgets.text("step_id","18233") # redshift2mysql prod
# dbutils.widgets.text("step_id","18234") # redshift2mysql uat
# dbutils.widgets.text("step_id","18266") # redshift2mysql prod
# dbutils.widgets.text("step_id","18266") # redshift2mysql prod
# dbutils.widgets.text("step_id","18267") # dms_redshift2mysql_agg_daily_incr_eligibility_snapshot_grp_rc_st_mvp_db_uat
# dbutils.widgets.text("mysql_env","")
# dbutils.widgets.text("secret_mgr_scope","de-scope-confg")
# dbutils.widgets.text("mysql_env","uat") # praveen env
# dbutils.widgets.text("step_id","18269") # praveen dmes
# dbutils.widgets.text("step_id","18275") # praveen dmes_rollup
# dbutils.widgets.text("step_id","18273") # fl testing from uat member_master.mrn_renewal_histories
# dbutils.widgets.text("step_id","18274") # fl testing from uat dmes
dbutils.widgets.text("secret_mgr_scope","de-scope-confg")

# dbutils.widgets.text("src_schema","de_currstate_report")
# dbutils.widgets.text("src_table","daily_member_eligibility_snapshot")
# dbutils.widgets.text("tgt_schema","data_science_edw_staging")
# dbutils.widgets.text("tgt_table","dbks_dmes_temp2")
#
# src_schema = dbutils.widgets.get("mysql_env")
# src_table = dbutils.widgets.get("src_table")

meta_step_id = dbutils.widgets.get("step_id")
# mysql_env = dbutils.widgets.get("mysql_env")
# mysql_env = dbutils.widgets.get("secret_mgr_scope")
secret_mgr_scope = dbutils.widgets.get("secret_mgr_scope")

In [ ]:
# redshift loader settings
# tempS3Dir = "s3://livongo-ds-temp"
# RS_SETUP_TEMP_S3IAM = 'arn:aws:iam::113931794823:role/Redshift_Databricks_S3'
# tgt_tbl="data_science_edw_staging.dbks_dmes_temp2"

In [ ]:
def get_current_notebook_path():
    notebook_path = dbutils.entry_point.getDbutils().notebook().getContext().notebookPath().get()
    return notebook_path

def get_environment_from_notebook_path():
    note_book_path = get_current_notebook_path()
    main_logger.info(f"Notebook path :{note_book_path}")
    if '/uat/' in note_book_path:
        return 'uat'
    elif '/prod/' in note_book_path:
        return 'prod'

def get_user_mysql_cred(mysql_env,secret_mgr_scope):
    if mysql_env.lower() in ['uat','prod']:
        mysql_ip_user = dbutils.secrets.get(scope=secret_mgr_scope, key="mysql_serv_acct")
        mysql_password = dbutils.secrets.get(scope=secret_mgr_scope, key="mysql_talend_password")
        return mysql_ip_user,mysql_password
    else:
        raise Exception(f"Invalid environment for mysql user credentials {mysql_env}")

def get_redshift_user_cred(redshift_env,secret_mgr_scope):
    if redshift_env.lower() in ['prod']:
        redshift_ip_user = dbutils.secrets.get(scope=secret_mgr_scope, key="redshift_talend_username")
        redshift_password = dbutils.secrets.get(scope=secret_mgr_scope, key="redshift_talend_password")
        return redshift_ip_user,redshift_password
    else:
        raise Exception(f"Invalid environment for redshift user credentials {mysql_env}")


In [ ]:
# df = ss.sql("select * from de_currstate_report.daily_member_eligibility_snapshot")
import entry_point
import importlib
from common.logmgr import get_logger

main_logger = get_logger()

importlib.reload(entry_point)
environment = get_environment_from_notebook_path()
main_logger.info(f"Environment identified as :{environment}")
mysql_user,mysql_pass = get_user_mysql_cred(mysql_env=environment,secret_mgr_scope=secret_mgr_scope)
redshift_user,redshift_pass = get_redshift_user_cred(redshift_env='prod',secret_mgr_scope=secret_mgr_scope)

if meta_step_id is None:
    raise Exception(f"Missing meta step id in environment :{environment}")
entry_point.start_dbks_dms(step_id=meta_step_id, meta_env=environment
                                     ,mysql_user=mysql_user, mysql_pass=mysql_pass
                                     ,redshift_user=redshift_user,redshift_pass=redshift_pass)